# Tutorial 4: Scheduling with CP: Some Basics 

Today we discover a new family of problems, called scheduling. Scheduling problems are widely present in real life applications such as timetabelling, transportation, project management, and manufacturing. We consider a particular problem called the job shop scheduling problem.  In this problem, we are given a set of $n$ jobs: $J_1, J_2, \ldots,  J_n$ and a set of $m$ machines $M_1, M_2, \ldots,  M_m$. 
Each job $J_i$ is defined by a set of $m$ (non-preemptive) tasks $T_{i,1} \ldots T_{i,m}$. Every task $T_{i,k}$ is associated with a duration $p_{i,k}$ and is supposed to be scheduled on machine $M_k$. 

The problem has two sets of constraints: 

 - Precedence constraints: Each job is associated with an order of tasks to respect when scheduling the different tasks.
 - Disjunctive constraints: Each machine can process only one task at a given time


The standard optimisation version of this problem asks to minimize the makespan, i.e., the total scheduling time.


Constraint programming has been widely and successfully used to solve scheduling problems. Many global constraints have been proposed. CP solvers often offer a dedicated library for scheduling. 
Please have a look at the diffrent features proposed in docplex here http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=scheduling#scheduling-functions 


In this tutorial, we will be using (only): 
 - Interval variables for the different variables of the problem:  http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html?highlight=interval_var#docplex.cp.expression.interval_var
 
 - end_before_start constraints to model precedence constraints : http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=end_before_start#docplex.cp.modeler.end_before_start

- no_overlap global constraint : http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=no_overlap#docplex.cp.modeler.no_overlap


The format for a job shop scheduling instance respects the following syntax: 
- The first line containts only $n$ $m$ in this order  ($n$ is the number of jobs and $m$ is the number of machines)
- Then $n$ lines are given. Each line $i$ is associated to the job $J_i$ and contains exactly $2 m$ integers $x^i_1$ $d^i_1$ $x^i_2$ $d^i_2$ $\ldots$ $x^i_n$ $d^i_n$. Each $x^i_k$ is the $k^{th}$ machine required by the $k^{th}$ task of the job $J_i$, and $d^i_k$ represents its duration. 

Consider for example the data file instance.data : 
    
 6  6
     
2   1  0   3  1   6  3   7  5   3  4   6

1   8  2   5  4  10  5  10  0  10  3   4

2   5  3   4  5   8  0   9  1   1  4   7
 
 1   5  0   5  2   5  3   3  4   8  5   9
 
 2   9  1   3  4   5  5   4  0   3  3   1
 
 1   3  3   3  5   9  0  10  4   4  2   1


This instance has $6$ jobs and $6$ machines. The first job requires the execution of task $T_{1,2}$ (on machine $2$) of duration $1$, 
then $T_{1,0}$ (on machine $0$) of duration $3$, etc. 

Write a simple python code to parse the data file instance.data

In [2]:
data = []

# Open the data file
with open("example.data", "r") as file:
    # Read the file line by line
    for line in file:
        # Strip any leading/trailing whitespace and split the line into individual values
        values = line.strip().split()
        
        # Convert each value to an integer and append it to the data list
        data.extend(map(int, values))

# Print the parsed data
print(data)


[6, 6, 2, 1, 0, 3, 1, 6, 3, 7, 5, 3, 4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9, 2, 9, 1, 3, 4, 5, 5, 4, 0, 3, 3, 1, 1, 3, 3, 3, 5, 9, 0, 10, 4, 4, 2, 1]


In [29]:
n, m = data[:2]

import numpy as np

jobs = []
task = np.zeros(m,n)
print(len(data))
for i in range(2, len(data)):
    for j in range(0, m):
        task.append(data[i*m+j])
    print(task)
    jobs.append(data[i:i+2])




# for i in range(2, len(data), 2):
#     print(data[i+m*(i-2):i+2*m])
#     jobs.append(data[i+m*(i-2):i+2*m])


print(jobs)


74
[4, 6, 1, 8, 2, 5]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9, 2, 9, 1, 3]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9, 2, 9, 1, 3, 4, 5, 5, 4, 0, 3]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9, 2, 9, 1, 3, 4, 5, 5, 4, 0, 3, 3, 1, 1, 3, 3, 3]
[4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9, 2, 9, 1, 3, 4, 5, 5, 4, 0, 3, 3, 1, 

IndexError: list index out of range

Create a matrix machine that satisfies: machine[i][k] is the machine of the $k^{th}$ task of job $i$ (one line)

In [9]:
# Create a matrix machine that satisfies: machine[i][k] is the machine of the kth task of job i (one line)

machine = [[jobs[i][2 * k] for k in range(m)] for i in range(n)]

# print(machine)
# Print the machine matrix
for row in machine:
    print(row)

[2, 0, 1, 3, 5, 4]
[0, 1, 3, 5, 4, 1]
[1, 3, 5, 4, 1, 2]
[3, 5, 4, 1, 2, 4]
[5, 4, 1, 2, 4, 5]
[4, 1, 2, 4, 5, 0]


Create a matrix duration that satisfies: duration[i][k] is the duration of the $k^{th}$ task of job $i$ (one line)

In [10]:
duration = [[jobs[i][2 * k + 1] for k in range(m)] for i in range(n)]

# Print the duration matrix
for row in duration:
    print(row)

[1, 3, 6, 7, 3, 6]
[3, 6, 7, 3, 6, 8]
[6, 7, 3, 6, 8, 5]
[7, 3, 6, 8, 5, 10]
[3, 6, 8, 5, 10, 10]
[6, 8, 5, 10, 10, 10]


Compute a naive upper bound for the makespan. Note: this upper bound will be used as an upper bound for every interval variable we create.

In [11]:
# Compute the naive upper bound (maximum sum of durations across all jobs)
naive_upper_bound = max(sum(jobs[i][2 * k + 1] for k in range(m)) for i in range(n))

# Print the naive upper bound
print("Naive Upper Bound:", naive_upper_bound)

Naive Upper Bound: 49


Create a CpoModel() and the different interval variables you need to solve this problem (don't forget to use the upper bound you computed earlier)

In [13]:
from config import setup
setup()

## install docplex first with $pip install docplex
from docplex.cp.model import *
from docplex.cp.config import get_default


In [17]:
def model(n):    
    # Create the CPO model
    model = CpoModel()

    # Create the interval variables
    tasks = [[interval_var(size=jobs[i][2 * k + 1], name="J{}-{}".format(i, k)) for k in range(m)] for i in range(n)]

    # Create the precedence constraints
    for i in range(n):
        for k in range(m - 1):
            model.add(end_before_start(tasks[i][k], tasks[i][k + 1]))

    return model, tasks

Post the precedence constraints using the end_before_start constraints 

In [ ]:
#Post the precedence constraints using end_before_start constraints

model, tasks = model(n)

# Create the objective function
model.add(maximize(sum(end_of(tasks[i][m - 1]) for i in range(n))))

# Solve the model
print("Solving model....")
msol = model.solve(TimeLimit=10)
print("Solution: ")
msol.print_solution()

# Create the CPO model
model = CpoModel()

# Create the interval variables


Post the disjunctive constraints using the no-overlap constraints 

Create a makespan interval variable and link it to some variables using the max global constraint
http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=max#docplex.cp.modeler.max

Add now the makespan as an objective function 

Solve this instance. What is the value of the makespan you found. You can print the solution in a format that is easy 
to see visually.  

Factorise your code so that it takes as input the data file and it solves the problem. Try to use few other instances https://github.com/tamy0612/JSPLIB/tree/master/instances
    
    


At this stage you are completly free to play. Try different instances, different configurations of the solver, different branching strategies, restarts, randomisation, etc. 

You may want to present your results as a cactus 🌵 plot : in the x-axis we have the runtime, on the y-axis, we have the number of instances solved. Also, some instances are still open in the literature. Have a look here for an up to date list of bounds http://optimizizer.com/TA.php

What did you learn loday? 